Création de la liste de tous les postes pour les Alpes. Ce cahier fournit les outils pour extraire l'information sur les postes météorologiques et nivologiques
de Météo-France.

Fonctions :
* extraire_information(département) : fournit l'information brute pour un département
* synthétiser_information(département) : fournit une liste synthétique (dataframe) des stations, à savoir :

  1. le nom, l'altitude, les coordonnées (latitude, longitude)
  2. les dates de début et de fin des mesures
  3. quel type (booléen) de variables a été mesuré :
    - pluie : pluie journalière
    - neige : cumul de neige sur 24 h
    - HN : épaisseur du manteau neigeux
    - TM : température moyenne
    - TN : température minimale
    - TX : température maximale
  
    export possible vers un fichier excel
* cartographier(liste des stations, département) : montre une carte de localisation des stations  

Les fonctions font appel à des variables globales : 
  * répertoire_travail : adresse du répertoire où les fichiers sont sauvegardés
  * export_détail, export_liste, export_carte: booléens d'export des fichiers bruts, de la liste synthétique des stations et de la carte
  * verbosity : booléen. Sert à afficher des messages de progression

  Comme Météo-France scinde ses fichiers départementaux en trois parties (avant 1950, de 1950 à la période récente, et la période comprenant l'année en cours et l'année), il faut télécharger les archives correspondantes, les décompresser et les assembler.

# Fonctions et initialisation

In [19]:
import pandas as pd
import os
import datetime
import json
import requests
import sys
import openpyxl #export xls
import numpy as np
import gzip
from tqdm import tqdm
import plotly.express as px

################################
## Auteur : Christoophe Ancey ##
## Date : Mai 2024            ##
## Mise à jour : juin 2025    ##
################################

# Création du répertoire de travail
# changer l'adresse si nécessaire
répertoire_travail = '/home/ancey/Météo-France/postes/'
date_calcul        = datetime.date.today()
année_en_cours     = date_calcul.year 


# exporte le détail de chaque département
export_détail = True
export_liste  = True
export_carte  = True
verbosity     = False

# adresse url de base
base_url = 	"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_"

# valeur à modifier si on veut afficher plus de lignes de dataframe
#pd.set_option('display.min_rows', 20)

# dictionnaire reliant numéro (ou abréviation) et nom
noms_département = dict()
noms_département['73'] = "Savoie"
noms_département['74'] = "Haute-Savoie"
noms_département['38'] = "Isère"
noms_département['05'] = "Hautes-Alpes"
noms_département['04'] = "Alpes-de-Haute-Provence"
noms_département['06'] = "Alpes-Maritimes"
noms_département['01'] = "Ain"
noms_département['25'] = "Doubs"
noms_département['26'] = "Drôme"
noms_département['09'] = "Ariège"
noms_département['31'] = "Haute-Garonne"
noms_département['64'] = "Pyrénées-Atlantiques"
noms_département['65'] = "Hautes-Pyrénées"
noms_département['66'] = "Pyrénées-Orientales"
noms_département['nord'] = "Alpes du Nord"
noms_département['sud'] = "Alpes du Sud"
noms_département['alpes'] = "Alpes"
noms_département['pyrénées'] = "Pyrénées"

# début des mesures au niveau départemental
an_début = {}
# pour les Alpes
an_début['01'] = '1852'
an_début['04'] = '1872'
an_début['05'] = '1877'
an_début['06'] = '1877'
an_début['25'] = '1852'
an_début['26'] = '1877'
an_début['38'] = '1872'
an_début['73'] = '1871'
an_début['74'] = '1876'
# pour les Pyrénées
an_début['09'] = '1872'
an_début['31'] = '1809'
an_début['64'] = '1865'
an_début['65'] = '1863'
an_début['66'] = '1850'
liste_départements = {'01','04','05','06','09','25','26','31','38','64','65','66','73','74'}
url_M1_département = {}
url_M2_département = {}
url_M3_département = {}
url_N1_département = {}
url_N2_département = {}
url_N3_département = {}

suffixe = '_latest-'+str(année_en_cours-1)+'-'+str(année_en_cours)
for département in liste_départements:
      url_N1_département[département] = base_url+département+"_"+an_début[département]+'-1949_autres-parametres.csv.gz'
      url_N2_département[département] = base_url+département+'_previous-1950-'+str(année_en_cours-2)+'_autres-parametres.csv.gz'
      #url_N3_département[département] = base_url+département+'_latest-2024-2025_autres-parametres.csv.gz'
      url_N3_département[département] = base_url+département+suffixe+'_autres-parametres.csv.gz'
      url_M1_département[département] = base_url+département+"_"+an_début[département]+'-1949_RR-T-Vent.csv.gz'
      url_M2_département[département] = base_url+département+'_previous-1950-'+str(année_en_cours-2)+'_RR-T-Vent.csv.gz'
      url_M3_département[département] = base_url+département+suffixe+'_RR-T-Vent.csv.gz'
      #url_M3_département[département] = base_url+département+'_latest-2024-2025_RR-T-Vent.csv.gz'


# teste si le répertoire existe 
if not os.path.exists(répertoire_travail):
    # crée le répertoire
    os.makedirs(répertoire_travail)
    print("J'ai créé le répertoire : ", répertoire_travail)
else:
    print("J'ai trouvé le répertoire : ",répertoire_travail)

# fonction pour télécharger les fichiers MF
def télécharger_fichier(url, nom_fichier, work_verbosity = True):
    fichier = os.path.join(répertoire_travail, nom_fichier) + '.gz'
    if os.path.isfile(fichier):
        if work_verbosity: print("Rien à faire. Le fichier est déjà présent.")
    else:
        requête_données = requests.get(url)
        if requête_données.status_code == 200:
            with open(fichier,mode='wb') as file:
             file.write(requête_données.content)
            if work_verbosity: print('Téléchargement : ', fichier)    
        else:
            print("Le lien ", url, "ne marche pas !")
            sys.exit()

# fonction pour décompresser les fichiers MF
def décompresser_archive(archive, work_verbosity = True):
    fichier             = os.path.join(répertoire_travail, archive) + '.gz'
    fichier_décompressé = os.path.join(répertoire_travail, archive) + '.csv'
    if os.path.isfile(fichier_décompressé):
        if work_verbosity: print("Rien à faire. Le fichier est déjà présent.")
    else:
        if os.path.exists(fichier):
            with gzip.open(fichier, mode='rb') as f_in:
                with open(fichier_décompressé, mode='wb') as f_out:
                    f_out.write(f_in.read())
            if work_verbosity: print('Archive décompressée : ', fichier)
        else:
            print("L'archive ", fichier, " n'est pas présente !")
            print("Téléchargez tout d'abord l'archive...")

######################################
# Extraction des informations brutes #
######################################
def extraction_information(dept: str):
    """ 
      pour chaque département, télécharge les fichiers de Météo France. On recherche :
            * les caractéristiques du poste, à savoir :
                  le nom, l'altitude, les coordonnées (latitude, longitude)
            * les dates de début et de fin des mesures
            * quel type de variables a été mesuré :
                  - pluie : pluie journalière
                  - neige : cumul de neige sur 24 h
                  - HN : épaisseur du manteau neigeux
                  - TM : température moyenne
                  - TN : température minimale
                  - TX : température maximale
            * le fichier d'origine de Météo-France (séquence) :
                  - 0 : météo (pluie, TM, TN, TX) avant 1950
                  - 1 : météo (pluie, TM, TN, TX) de 1950 à A-2
                  - 2 : météo (pluie, TM, TN, TX) après A-1
                  - 3 : nivo (neige, HN) avant 1950
                  - 4 : nivo (neige, HN) de 1950 à A-2
                  - 5 : nivo (neige, HN) après A-1
                  où A désigne l'année en cours
      La fonction fournit un fichier "fichier_postes"  (dataframe).
      Les données sont exportées si la variable globale export_détail = True.
    """
    fichier_postes = []
    liste_url = [
        url_M1_département[dept], url_M2_département[dept], url_M3_département[dept],
        url_N1_département[dept], url_N2_département[dept], url_N3_département[dept]
    ]
    
    # structure du dataframe
    base_columns = ['poste', 'nom', 'valeurs', 'alt', 'lat', 'lon', 
                   'début', 'fin', 'pluie', 'TM', 'TN', 'TX', 'HN', 'neige', 'séquence']
    
    for indice_fichier in tqdm(range(6)):
        archive = f'test_{indice_fichier}_{dept}'
        télécharger_fichier(liste_url[indice_fichier], archive, work_verbosity=verbosity)
        décompresser_archive(archive, work_verbosity=verbosity)
        
        fichier = os.path.join(répertoire_travail, f'{archive}.csv')
        données_totales = pd.read_csv(
            fichier, sep=";", encoding='utf-8',
            parse_dates=["AAAAMMJJ"], dtype={"NUM_POSTE": str}
        )
        données_totales['NUM_POSTE'] = données_totales['NUM_POSTE'].str.strip()
        
        # Initialisation
        df_poste = pd.DataFrame(columns=base_columns)
        
        # nombre de données par station
        station_counts         = données_totales['NUM_POSTE'].value_counts().reset_index()
        station_counts.columns = ["poste", "nb valeurs"]
        
        for _, row in station_counts.iterrows():
            numéro_poste = row['poste']
            station_data = données_totales[données_totales['NUM_POSTE'] == numéro_poste]
            
            première_ligne = station_data.iloc[0]
            dernière_ligne = station_data.iloc[-1]
            
            # vérifie si les données sont disponibles
            if indice_fichier < 3:  # # Données météorologiques
                data_flags = {
                    'pluie': station_data['RR'].notna().any(),
                    'TM': station_data['TM'].notna().any(),
                    'TN': station_data['TN'].notna().any(),
                    'TX': station_data['TX'].notna().any(),
                    'HN': False,
                    'neige': False
                }
            else:  # neige
                data_flags = {
                    'pluie': False,
                    'TM': False,
                    'TN': False,
                    'TX': False,
                    'HN': station_data['HNEIGEF'].notna().any(),
                    'neige': station_data['NEIGETOTX'].notna().any()
                }
            
            station_info = {
                'poste': numéro_poste,
                'nom': première_ligne['NOM_USUEL'].title(),
                'valeurs': row['nb valeurs'],
                'alt': première_ligne['ALTI'],
                'lat': première_ligne['LAT'],
                'lon': première_ligne['LON'],
                'début': première_ligne['AAAAMMJJ'].strftime("%Y-%m-%d"),
                'fin': dernière_ligne['AAAAMMJJ'].strftime("%Y-%m-%d"),
                'séquence': indice_fichier,
                **data_flags
            }
            
            # Append using loc to avoid concatenation warnings
            df_poste.loc[len(df_poste)] = station_info
        
        fichier_postes.append(df_poste)
    
    # Concaténation
    result = pd.concat(fichier_postes).sort_values('poste')
    
    if export_détail:
        output_file = os.path.join(répertoire_travail, f'liste_détaillée_{dept}.xlsx')
        result.to_excel(output_file, index=False, sheet_name='liste')
    
    return result


######################################
# Synthèse des informations brutes   #
######################################
def synthétiser_information(dept : str):
      """ 
      fournit une synthèse de l'information pour le département, à savoir :
            * le nom, l'altitude, les coordonnées (latitude, longitude)
            * les dates de début et de fin des mesures
            * quel type de variables a été mesuré :
                  - pluie : pluie journalière
                  - neige : cumul de neige sur 24 h
                  - HN : épaisseur du manteau neigeux
                  - TM : température moyenne
                  - TN : température minimale
                  - TX : température maximale
      La fonction fait appel à la fonction extraction_information
      La fonction fournit un fichier "liste_finale" (dataframe).
      Les données sont exportées si la variable globale export_liste = True.
      """
      fichier_postes = extraction_information(dept)
      liste_poste = np.array( fichier_postes['poste'].value_counts().reset_index())[:,0]
      liste_finale = pd.DataFrame(columns=['poste', 'nom', 'valeurs', 'alt', 'lat', 'lon', 
                                                  'début','fin', 'pluie', 'neige','HN','TM', 'TN', 'TX' ] )
      liste_finale = pd.DataFrame({
            'poste': pd.Series(dtype='object'),
            'nom': pd.Series(dtype='object'),
            'valeurs': pd.Series(dtype='int'),
            'alt': pd.Series(dtype='float'),
            'lat': pd.Series(dtype='float'),
            'lon': pd.Series(dtype='float'),
            'début': pd.Series(dtype='datetime64[ns]'),
            'fin': pd.Series(dtype='datetime64[ns]'),
            'pluie': pd.Series(dtype='bool'),
            'neige': pd.Series(dtype='bool'),
            'HN': pd.Series(dtype='bool'),
            'TM': pd.Series(dtype='bool'),
            'TN': pd.Series(dtype='bool'),
            'TX': pd.Series(dtype='bool')
        })
      # 
      for poste in liste_poste:
            sélection_valeurs = fichier_postes[fichier_postes['poste']==poste]
            nombre_valeurs = sélection_valeurs['valeurs'].sum()
            numéro_poste   = poste
            première_ligne = sélection_valeurs.iloc[0]
            nom_poste = première_ligne['nom']
            latitude  = première_ligne['lat']
            longitude = première_ligne['lon']
            altitude  = première_ligne['alt']
            date_début= sélection_valeurs['début'].min()
            date_fin  = sélection_valeurs['fin'].max()
            existe_pluie = sélection_valeurs['pluie'].any()
            existe_neige = sélection_valeurs['neige'].any()
            existe_HN = sélection_valeurs['HN'].any()
            existe_TN = sélection_valeurs['TN'].any()
            existe_TM = sélection_valeurs['TM'].any()
            existe_TX = sélection_valeurs['TX'].any()
            dict1 = {'poste':[numéro_poste], 'nom':[nom_poste], 'valeurs':[nombre_valeurs], 
                          'alt':[altitude], 'lat':[latitude],'lon':[longitude],
                          'début':[date_début], 'fin':[date_fin], 'pluie':[existe_pluie],
                          'TM':[existe_TM],'TN':[existe_TN],'TX':[existe_TX],
                          'HN':[existe_HN],'neige':[existe_neige]} 
            ajout     = pd.DataFrame(dict1)
            liste_finale = pd.concat([liste_finale, ajout])
      liste_finale = liste_finale.sort_values(by=['nom'])
      # export de la synthèse du département
      if export_liste:
            fichier_excel   = os.path.join(répertoire_travail, 'liste_synthétique_'+dept+'.xlsx')
            onglet_liste = 'liste-stations'
            liste_finale.to_excel(fichier_excel, index=False, sheet_name= onglet_liste)
      return liste_finale

######################################
# Carte des stations                 #
######################################
def cartographier_postes(liste_p, dept: str):
      """ 
      Cartographie les postes.
      La carte est exportée si la variable globale export_carte = True.
      """
      m_lat=np.mean(np.array(liste_p.lat))
      m_lon=np.mean(np.array(liste_p.lon))
      aujourdhui = datetime.datetime.now()
      title = noms_département[dept]+" : postes actifs en " + aujourdhui.strftime("%Y")
      fig = px.scatter_mapbox(liste_p, 'lat', lon= 'lon', 
                        hover_name= "nom", hover_data= ['poste','alt', 'début', 'fin'], 
                        zoom=5, height= 800, width= 900, 
                        )
      fig.update_layout(title_text= title, title_x=0.5  , hoverlabel= dict(bgcolor= 'rgb(255,255,255)'))
      fig.update_traces(marker=dict(color='red'))
      #===================== Ajoute une couche des départements français à partir d'un fichier GeoJSON
      fig.update_layout(mapbox_layers=[
            {
            "sourcetype": "geojson",
            # "source": "https://france-geojson.gregoiredavid.fr/repo/departements.geojson",
            "source": "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb", 
            "type": "line",
            "color": "rgba(0, 0, 0, 0.9)",
            "line": {"width": 1}
            }
            ])
      fig.update_layout(mapbox_style= "open-street-map", mapbox_zoom=8, mapbox_center = {"lat": m_lat, "lon": m_lon})
      fig.show()
      if export_carte:
            # Export PNG
            file_png= os.path.join(répertoire_travail, 'carte_postes_'+dept + '.png')
            fig.write_image(file_png, height= 800, width= 900,   scale = 3)


J'ai trouvé le répertoire :  /home/ancey/Météo-France/postes/


# Extraction pour la Savoie

In [20]:
liste_73 = synthétiser_information('73')
cartographier_postes(liste_73, dept = '73')

100%|██████████| 6/6 [00:59<00:00,  9.85s/it]


# Extraction Alpes du Nord

In [ ]:
départements_alpes_nord = ['73','74','38']


      
# initialisation
liste_dataframes = []

for dept in départements_alpes_nord:
    liste_départementale = synthétiser_information(dept)
    liste_dataframes.append(liste_départementale)  # Add to list

# Concaténation
liste_alpes_nord = pd.concat(liste_dataframes, ignore_index=True)
cartographier_postes(liste_alpes_nord, dept = 'nord')
      

100%|██████████| 6/6 [00:49<00:00,  8.19s/it]


In [8]:
liste_alpes_nord

,poste,nom,valeurs,alt,lat,lon,début,fin,pluie,neige,HN,TM,TN,TX
0,73001001,Aiguebelette,5539,400,45.551667,5.813333,1941-10-01,1966-08-31,True,False,False,False,False,False
0,73004001,Aillon Le Jeune,46217,900,45.617833,6.080167,1934-06-01,2020-12-31,True,True,True,False,False,False
0,73008003,Aix Les Bains,5296,234,45.700333,5.889667,2003-01-01,2015-08-31,True,False,False,False,False,False
0,73008001,Aix-Les-Bains,9048,365,45.693333,5.908333,1947-01-01,1971-01-31,True,False,False,False,True,True
0,73011002,Albertville,43171,331,45.662167,6.371833,1948-01-01,2015-08-31,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,38548002,Villard-De-Lans-Gend,19607,1031,45.070333,5.552333,1916-01-01,1969-04-30,True,False,False,False,True,True
0,38549001,Villard-Notre-Dame Village,5313,1550,45.018333,6.041667,1950-04-18,1965-11-30,True,False,False,False,False,False
0,38559004,Vinay,656,400,45.212000,5.376833,2015-03-01,2016-01-31,True,True,True,False,False,False
0,38559001,Vinay Bourg,1952,260,45.210000,5.401667,1949-01-01,1954-03-31,True,False,False,False,False,False


In [23]:
fichier_excel   = os.path.join(répertoire_travail, 'liste_synthétique_Alpes_Nord.xlsx')
onglet_liste = 'liste-stations'
liste_alpes_nord.to_excel(fichier_excel, index=False, sheet_name= onglet_liste)

# Extraction pour les Hautes Alpes

In [24]:
liste_05 = synthétiser_information('05')
cartographier_postes(liste_05, dept = '05')

100%|██████████| 6/6 [00:27<00:00,  4.54s/it]


In [11]:
liste_05

,poste,nom,valeurs,alt,lat,lon,début,fin,pluie,neige,HN,TM,TN,TX
0,05001001,Abries,36317,1560,44.796667,6.928333,1932-01-01,2025-05-31,True,False,False,False,True,True
0,05001401,Abries -S-,1,2070,44.791333,6.949667,2007-01-01,2007-01-01,False,False,False,False,False,False
0,05001400,Abries_Nivo,5830,2018,44.792833,6.947000,1985-12-23,2025-03-22,True,True,True,False,True,True
0,05139006,Agnieres-En-Devoluy,52188,1244,44.696167,5.880833,1928-11-01,2025-06-13,True,True,True,True,True,True
0,05004001,Ancelle,48053,1354,44.624500,6.209833,1928-03-01,2025-06-13,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,05181002,Villar D'Arene,15211,1665,45.030833,6.361667,2004-07-31,2025-06-13,True,True,False,True,True,True
0,05182001,Villar Loubiere,48319,1072,44.826833,6.146000,1928-01-01,2025-06-13,True,True,True,True,True,True
0,05183001,Villar St Pancrace,15976,1310,44.880333,6.640333,2003-08-01,2025-06-13,True,True,True,True,True,True
0,05181001,Villar-D'Arene Village,5586,1650,45.041667,6.336667,1944-06-01,1967-01-31,True,False,False,False,False,False


# Création pour les Alpes du Sud

In [25]:
départements_alpes_sud = ['05','04','06']


 
# initialisation
liste_dataframes = []

for dept in départements_alpes_sud:
    liste_départementale = synthétiser_information(dept)
    liste_dataframes.append(liste_départementale)   

# Concaténation
liste_alpes_sud = pd.concat(liste_dataframes, ignore_index=True)
cartographier_postes(liste_alpes_sud, dept = 'sud')

100%|██████████| 6/6 [00:42<00:00,  7.17s/it]


In [13]:
fichier_excel   = os.path.join(répertoire_travail, 'liste_synthétique_Alpes_Sud.xlsx')
onglet_liste = 'liste-stations'
liste_alpes_sud.to_excel(fichier_excel, index=False, sheet_name= onglet_liste)

In [14]:
extraction_information('05')

100%|██████████| 6/6 [00:26<00:00,  4.35s/it]


,poste,nom,valeurs,alt,lat,lon,début,fin,pluie,TM,TN,TX,HN,neige,séquence
18,05001001,Abries,40,1560,44.796667,6.928333,1947-01-05,1949-01-08,False,False,False,False,False,False,3
57,05001001,Abries,4355,1560,44.796667,6.928333,1950-05-17,2023-02-28,False,False,False,False,False,False,4
31,05001001,Abries,517,1560,44.796667,6.928333,2024-01-01,2025-05-31,True,False,True,True,False,False,2
17,05001001,Abries,26232,1560,44.796667,6.928333,1950-05-01,2023-12-31,True,False,True,True,False,False,1
16,05001001,Abries,5173,1560,44.796667,6.928333,1932-01-01,1949-05-31,True,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,05183001,Villar St Pancrace,7458,1310,44.880333,6.640333,2003-08-01,2023-12-31,True,True,True,True,False,False,1
1,05183001,Villar St Pancrace,530,1310,44.880333,6.640333,2024-01-01,2025-06-13,True,True,True,True,False,False,2
32,05183001,Villar St Pancrace,7458,1310,44.880333,6.640333,2003-08-01,2023-12-31,False,False,False,False,True,True,4
67,05184002,Vitrolles,5813,595,44.408667,5.951667,1997-02-01,2012-12-31,True,True,True,True,False,False,1


# Extraction pour les Pyrénées

In [26]:
départements_pyrénées = ['09','31','64','65','66']


# initialisation
liste_dataframes = []

for dept in départements_pyrénées:
    liste_départementale = synthétiser_information(dept)
    liste_dataframes.append(liste_départementale)   

# Concaténation
liste_pyrénées = pd.concat(liste_dataframes, ignore_index=True)
cartographier_postes(liste_pyrénées, dept = 'pyrénées')
      

100%|██████████| 6/6 [00:40<00:00,  6.67s/it]


In [29]:
fichier_excel   = os.path.join(répertoire_travail, 'liste_synthétique_Pyrénées.xlsx')
onglet_liste = 'liste-stations'
liste_pyrénées.to_excel(fichier_excel, index=False, sheet_name= onglet_liste)